<a href="https://colab.research.google.com/github/kwonsungja/app_nouns/blob/main/app_nouns_regular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gradio as gr
import pandas as pd
import random

# Load the CSV file from the provided URL
csv_url = "https://github.com/kwonsungja/app_nouns/blob/main/regular_Nouns_task.csv"
df = pd.read_csv(csv_url)

# Count the number of items in each level
cefr_counts = df["level"].value_counts()
cefr_levels_with_counts = [
    f"{level} ({level_counts.get(level, 0)} items)"
    for level in ["s", "es", "ies", "es"]
]

# Function to initialize user-specific state
def initialize_user_state():
    return {
        "remaining_nouns": pd.DataFrame(),
        "current_level": None,
        "score": 0,
        "trials": 0,
        "current_index": -1,
        "level_scores": {level: {"score": 0, "trials": 0} for level in ["s", "es", "ies", "es"]},
    }

# Function to filter nouns by level if the level changes
def filter_nouns_if_needed(cefr_level_with_count, user_state):
    cefr_level = cefr_level_with_count.split(" ")[0]  # Extract level (e.g., "A1" from "A1 (10 items)")
    if user_state["current_level"] != level:
        filtered_nouns = df[df["level"] == level].copy()
        if filtered_nouns.empty:
            return user_state, f"No nouns available for the Level: {level}. Please select a different level."
        # Update the state
        user_state["remaining_nouns"] = filtered_nouns
        user_state["current_level"] = level
        user_state["score"] = 0
        user_state["trials"] = 0
        user_state["current_index"] = -1
        return user_state, f"Level {cefr_level} selected. Click 'Show the Noun' to start!"
    # If the level hasn't changed, do nothing
    return user_state, None

# Function to show the next noun
def show_next_noun(level_with_count, user_state):
    user_state, feedback = filter_nouns_if_needed(level_with_count, user_state)  # Filter nouns if needed
    if user_state["remaining_nouns"].empty:
        return user_state, feedback or "All nouns have been answered correctly. Great job!", ""
    user_state["current_index"] = random.randint(0, len(user_state["remaining_nouns"]) - 1)
    selected_noun = user_state["remaining_nouns"].iloc[user_state["current_index"]]
    return user_state, f"What's the plural form of '{selected_noun['Singular']}'?", ""

# Function to check the user's answer
def check_plural(user_plural, user_state):
    if user_state["remaining_nouns"].empty:
        return user_state, "All nouns have been answered correctly. Great job! (Score: {user_state['score']}/{user_state['trials']})"

    index = user_state["current_index"]
    if index == -1:
        return user_state, "Please click 'Show the Noun' first. (Score: {user_state['score']}/{user_state['trials']})"

    noun_data = user_state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    user_state["trials"] += 1
    user_state["level_scores"][user_state["current_level"]]["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        user_state["score"] += 1
        user_state["level_scores"][user_state["current_level"]]["score"] += 1
        feedback = f"✅ Correct! '{correct_plural}' is the plural form of '{singular}'. Click 'Show the Noun' to continue."
        # Remove the correctly answered noun
        user_state["remaining_nouns"] = user_state["remaining_nouns"].drop(user_state["remaining_nouns"].index[index])
    else:
        feedback = f"❌ Incorrect. The correct plural form is '{correct_plural}' for '{singular}'. It will appear again."

    if user_state["remaining_nouns"].empty:
        feedback += f"\n🎉 All nouns have been answered correctly. Great job! (Score: {user_state['score']}/{user_state['trials']})"

    return user_state, f"{feedback} (Score: {user_state['score']}/{user_state['trials']})"

# Function to display the total score
def display_total_score(user_state):
    total_score = ", ".join(
        f"{level}({user_state['level_scores'][level]['score']}/{user_state['level_scores'][level]['trials']})"
        for level in ["s", "es", "ies", "es"]
    )
    return total_score

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# NounSmart: Practice regular Plural Nouns")
    gr.Markdown("""
    ## How to Use the App
    1. **Follow the steps from Step 1 to Step 4.**
    2. **Click 'Show Report' to view overall feedback across all levels.**
    """)

    # Dropdown for level selection with item counts
    cefr_dropdown = gr.Dropdown(
        label="Step 1. Select a Level to start.",
        choices=cefr_levels_with_counts,
        value=cefr_levels_with_counts[0],
        interactive=True
    )

    # Button to display a noun
    show_button = gr.Button("Step 2. Show the Noun")
    noun_display = gr.Textbox(label="Singular Noun", value="Select a level to start", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Step 3. Type Your answer.", placeholder="Type your answer here")

    # Submit button and combined feedback + score display
    submit_button = gr.Button("Step 4. See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback and Score", interactive=False)

    # Total score display
    total_score_button = gr.Button("Show Report")
    total_score_display = gr.Textbox(label="Total Score by Levels", interactive=False)

    # Initialize state
    state = gr.State(initialize_user_state())

    # Button click actions
    show_button.click(fn=show_next_noun, inputs=[cefr_dropdown, state], outputs=[state, noun_display, plural_input])
    submit_button.click(fn=check_plural, inputs=[plural_input, state], outputs=[state, feedback_display])
    total_score_button.click(fn=display_total_score, inputs=state, outputs=total_score_display)

app.launch()

ModuleNotFoundError: No module named 'gradio'